# In the name of Allah

In [28]:
!pip install pyspark
!pip install parsivar

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType
import pandas as pd
import numpy as np

In [30]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/Colab Notebooks/Divar/DMC-phase1-DataSets-20211225T055615Z-001.zip" 

path = "/content/DMC-phase1-DataSets"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/Colab Notebooks/Divar/DMC-phase1-DataSets-20211225T055615Z-001.zip
replace DMC-phase1-DataSets/reject_reasons_info.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace DMC-phase1-DataSets/DMC-Phase1-Validation.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace DMC-phase1-DataSets/DMC-Train.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [31]:
spark = SparkSession.builder.appName('Divar').getOrCreate()

df = spark.read.parquet(path + '/DMC-Train.parquet')

df = df.drop("reject_reason_id")

df = df.withColumn("review_label", when(df.review_label == "accept","true")\
      .when(df.review_label == "reject","false"))

In [ ]:
df.show()

+--------------------+--------------------+------------+
|             post_id|           post_data|review_label|
+--------------------+--------------------+------------+
|cb000599-2ee2-42c...|{"body_status": "...|        true|
|12063741-6634-444...|{"body_status": "...|       false|
|81c93119-5c06-412...|{"body_status": "...|        true|
|b5a5bfa7-03be-408...|{"brand": "\u0633...|        true|
|3414e920-dfaf-44a...|{"body_status": "...|       false|
|10aa8a3b-e465-45c...|{"body_status": "...|       false|
|79f05181-ddfd-44b...|{"body_status": "...|        true|
|3845f74f-6731-44f...|{"body_status": "...|        true|
|6fdbdc90-dc85-49e...|{"body_status": "...|       false|
|c8ba52f7-7025-4f8...|{"body_status": "...|        true|
|58af8fd6-3ef9-483...|{"body_status": "...|        true|
|389f6987-8cb8-48a...|{"body_status": "...|        true|
|07e64f4a-9767-435...|{"brand": "\u067e...|        true|
|df968fd4-fb04-427...|{"body_status": "...|        true|
|301684eb-b556-482...|{"body_st

In [ ]:
df.head()

Row(post_id='cb000599-2ee2-42c1-9f0e-32cfeb940398', post_data='{"body_status": "witout-color", "brand": "\\u062a\\u06cc\\u0628\\u0627::Tiba", "brand_model": "Tiba Sedan SX", "category": "light", "color": "\\u0633\\u0641\\u06cc\\u062f", "description": "\\u0628\\u062f\\u0648\\u0646 \\u0631\\u0646\\u06af\\u060c\\u06a9\\u0645 \\u06a9\\u0627\\u0631\\u06a9\\u0631\\u062f", "document": "single-sheet", "gearbox": "manual", "new_price": 103000000, "post_type": "\\u0641\\u0631\\u0648\\u0634\\u06cc", "selling_type": "cash", "third_party_insurance_deadline": "5", "title": "\\u062a\\u06cc\\u0628\\u0627 \\u0635\\u0646\\u062f\\u0648\\u0642\\u200c\\u062f\\u0627\\u0631 SX\\u060c \\u0645\\u062f\\u0644 \\u06f1\\u06f3\\u06f9\\u06f7", "usage": 23000, "year": "\\u06f1\\u06f3\\u06f9\\u06f7"}', review_label='true')

In [ ]:
import json
newJson = '''{"body_status": "witout-color", "brand": "\\u062a\\u06cc\\u0628\\u0627::Tiba", "brand_model": "Tiba Sedan SX", "category": "light", "color": "\\u0633\\u0641\\u06cc\\u062f", "description": "\\u0628\\u062f\\u0648\\u0646 \\u0631\\u0646\\u06af\\u060c\\u06a9\\u0645 \\u06a9\\u0627\\u0631\\u06a9\\u0631\\u062f", "document": "single-sheet", "gearbox": "manual", "new_price": 103000000, "post_type": "\\u0641\\u0631\\u0648\\u0634\\u06cc", "selling_type": "cash", "third_party_insurance_deadline": "5", "title": "\\u062a\\u06cc\\u0628\\u0627 \\u0635\\u0646\\u062f\\u0648\\u0642\\u200c\\u062f\\u0627\\u0631 SX\\u060c \\u0645\\u062f\\u0644 \\u06f1\\u06f3\\u06f9\\u06f7", "usage": 23000, "year": "\\u06f1\\u06f3\\u06f9\\u06f7"}'''
pdf = json. loads(newJson)
json. loads(newJson)

{'body_status': 'witout-color',
 'brand': 'تیبا::Tiba',
 'brand_model': 'Tiba Sedan SX',
 'category': 'light',
 'color': 'سفید',
 'description': 'بدون رنگ،کم کارکرد',
 'document': 'single-sheet',
 'gearbox': 'manual',
 'new_price': 103000000,
 'post_type': 'فروشی',
 'selling_type': 'cash',
 'third_party_insurance_deadline': '5',
 'title': 'تیبا صندوق\u200cدار SX، مدل ۱۳۹۷',
 'usage': 23000,
 'year': '۱۳۹۷'}

In [ ]:
schema = StructType([ 
    StructField("body_status",StringType(),True), 
    StructField("brand",StringType(),True), 
    StructField("brand_model",StringType(),True), 
    StructField("category",StringType(),True), 
    StructField("color",StringType(),True), 
    StructField("description",StringType(),True), 
    StructField("document",StringType(),True), 
    StructField("gearbox",StringType(),True), 
    StructField("new_price",StringType(),True), 
    StructField("post_type",StringType(),True), 
    StructField("selling_type",StringType(),True), 
    StructField("third_party_insurance_deadline",StringType(),True), 
    StructField("title",StringType(),True), 
    StructField("usage", StringType(), True),
    StructField("year", StringType(), True)
  ])

df = df.withColumn("post_data", from_json("post_data", schema))\
    .select(col('post_id'),
            col('review_label'),
            col('post_data.*'))

In [ ]:
df.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- review_label: string (nullable = true)
 |-- body_status: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- brand_model: string (nullable = true)
 |-- category: string (nullable = true)
 |-- color: string (nullable = true)
 |-- description: string (nullable = true)
 |-- document: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- new_price: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- selling_type: string (nullable = true)
 |-- third_party_insurance_deadline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- usage: string (nullable = true)
 |-- year: string (nullable = true)



In [ ]:
df = df.drop('brand')
df.show()

+--------------------+------------+--------------------+--------------------+--------+---------+--------------------+--------------+-------+---------+---------+---------------+------------------------------+--------------------+------+----+
|             post_id|review_label|         body_status|         brand_model|category|    color|         description|      document|gearbox|new_price|post_type|   selling_type|third_party_insurance_deadline|               title| usage|year|
+--------------------+------------+--------------------+--------------------+--------+---------+--------------------+--------------+-------+---------+---------+---------------+------------------------------+--------------------+------+----+
|cb000599-2ee2-42c...|        true|        witout-color|       Tiba Sedan SX|   light|     سفید|  بدون رنگ،کم کارکرد|  single-sheet| manual|103000000|    فروشی|           cash|                             5|تیبا صندوق‌دار SX...| 23000|۱۳۹۷|
|12063741-6634-444...|       false| 

In [ ]:
### First code about counting null values and second is about see which column that you want ###

# df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show() 
# df.select(['third_party_insurance_deadline','usage','review_label','description']).show()  
# df.na.fill('',['color']).show()

# Text proccecing

## Utils

In [ ]:
from __future__ import unicode_literals

import collections
import gc
import json
import re
import os

import numpy as np
import pandas as pd
import parsivar
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [ ]:
def read_json(path, n_lines_to_read=None):
    """
    read json file line by line iteratively (generator function).
    use this function to read json files when you have memory limitations.
    """

    with open(path) as f:
        for i, line in enumerate(tqdm(f)):
            if n_lines_to_read == i:
                break
            yield json.loads(line)

In [ ]:
parsivar_normalizer = parsivar.Normalizer(statistical_space_correction=True)

char_mappings = {
    "٥": "5",
    "А": "a",
    "В": "b",
    "Е": "e",
    "Н": "h",
    "Р": "P",
    "С": "C",
    "Т": "T",
    "а": "a",
    "г": "r",
    "е": "e",
    "к": "k",
    "м": "m",
    "о": "o",
    "р": "p",
    "ڈ": "د",
    "ڇ": "چ",
    # Persian numbers (will be raplaced by english one)
    "۰": "0",
    "۱": "1",
    "۲": "2",
    "۳": "3",
    "۴": "4",
    "۵": "5",
    "۶": "6",
    "۷": "7",
    "۸": "8",
    "۹": "9",
    ".": ".",
    # Arabic numbers (will be raplaced by english one)
    "٠": "0",
    "١": "1",
    "٢": "2",
    "٣": "3",
    "٤": "4",
    "٥": "5",
    "٦": "6",
    "٧": "7",
    "٨": "8",
    "٩": "9",
    # Special Arabic Characters (will be replaced by persian one)
    "ك": "ک",
    "ى": "ی",
    "ي": "ی",
    "ؤ": "و",
    "ئ": "ی",
    "إ": "ا",
    "أ": "ا",
    "آ": "ا",
    "ة": "ه",
    "ء": "ی",
    # French alphabet (will be raplaced by english one)
    "à": "a",
    "ä": "a",
    "ç": "c",
    "é": "e",
    "è": "e",
    "ê": "e",
    "ë": "e",
    "î": "i",
    "ï": "i",
    "ô": "o",
    "ù": "u",
    "û": "u",
    "ü": "u",
    # Camma (will be replace by dots for floating point numbers)
    ",": ".",
    # And (will be replace by dots for floating point numbers)
    "&": " and ",
    # Vowels (will be removed)
    "ّ": "",  # tashdid
    "َ": "",  # a
    "ِ": "",  # e
    "ُ": "",  # o
    "ـ": "",  # tatvil
    # Spaces
    "‍": "",  # 0x9E -> ZERO WIDTH JOINER
    "‌": " ",  # 0x9D -> ZERO WIDTH NON-JOINER
    # Arabic Presentation Forms-A (will be replaced by persian one)
    "ﭐ": "ا",
    "ﭑ": "ا",
    "ﭖ": "پ",
    "ﭗ": "پ",
    "ﭘ": "پ",
    "ﭙ": "پ",
    "ﭞ": "ت",
    "ﭟ": "ت",
    "ﭠ": "ت",
    "ﭡ": "ت",
    "ﭺ": "چ",
    "ﭻ": "چ",
    "ﭼ": "چ",
    "ﭽ": "چ",
    "ﮊ": "ژ",
    "ﮋ": "ژ",
    "ﮎ": "ک",
    "ﮏ": "ک",
    "ﮐ": "ک",
    "ﮑ": "ک",
    "ﮒ": "گ",
    "ﮓ": "گ",
    "ﮔ": "گ",
    "ﮕ": "گ",
    "ﮤ": "ه",
    "ﮥ": "ه",
    "ﮦ": "ه",
    "ﮪ": "ه",
    "ﮫ": "ه",
    "ﮬ": "ه",
    "ﮭ": "ه",
    "ﮮ": "ی",
    "ﮯ": "ی",
    "ﮰ": "ی",
    "ﮱ": "ی",
    "ﯼ": "ی",
    "ﯽ": "ی",
    "ﯾ": "ی",
    "ﯿ": "ی",
    # Arabic Presentation Forms-B (will be removed)
    "ﹰ": "",
    "ﹱ": "",
    "ﹲ": "",
    "ﹳ": "",
    "ﹴ": "",
    "﹵": "",
    "ﹶ": "",
    "ﹷ": "",
    "ﹸ": "",
    "ﹹ": "",
    "ﹺ": "",
    "ﹻ": "",
    "ﹼ": "",
    "ﹽ": "",
    "ﹾ": "",
    "ﹿ": "",
    # Arabic Presentation Forms-B (will be replaced by persian one)
    "ﺀ": "ی",
    "ﺁ": "ا",
    "ﺂ": "ا",
    "ﺃ": "ا",
    "ﺄ": "ا",
    "ﺅ": "و",
    "ﺆ": "و",
    "ﺇ": "ا",
    "ﺈ": "ا",
    "ﺉ": "ی",
    "ﺊ": "ی",
    "ﺋ": "ی",
    "ﺌ": "ی",
    "ﺍ": "ا",
    "ﺎ": "ا",
    "ﺏ": "ب",
    "ﺐ": "ب",
    "ﺑ": "ب",
    "ﺒ": "ب",
    "ﺓ": "ه",
    "ﺔ": "ه",
    "ﺕ": "ت",
    "ﺖ": "ت",
    "ﺗ": "ت",
    "ﺘ": "ت",
    "ﺙ": "ث",
    "ﺚ": "ث",
    "ﺛ": "ث",
    "ﺜ": "ث",
    "ﺝ": "ج",
    "ﺞ": "ج",
    "ﺟ": "ج",
    "ﺠ": "ج",
    "ﺡ": "ح",
    "ﺢ": "ح",
    "ﺣ": "ح",
    "ﺤ": "ح",
    "ﺥ": "خ",
    "ﺦ": "خ",
    "ﺧ": "خ",
    "ﺨ": "خ",
    "ﺩ": "د",
    "ﺪ": "د",
    "ﺫ": "ذ",
    "ﺬ": "ذ",
    "ﺭ": "ر",
    "ﺮ": "ر",
    "ﺯ": "ز",
    "ﺰ": "ز",
    "ﺱ": "س",
    "ﺲ": "س",
    "ﺳ": "س",
    "ﺴ": "س",
    "ﺵ": "ش",
    "ﺶ": "ش",
    "ﺷ": "ش",
    "ﺸ": "ش",
    "ﺹ": "ص",
    "ﺺ": "ص",
    "ﺻ": "ص",
    "ﺼ": "ص",
    "ﺽ": "ض",
    "ﺾ": "ض",
    "ﺿ": "ض",
    "ﻀ": "ض",
    "ﻁ": "ط",
    "ﻂ": "ط",
    "ﻃ": "ط",
    "ﻄ": "ط",
    "ﻅ": "ظ",
    "ﻆ": "ظ",
    "ﻇ": "ظ",
    "ﻈ": "ظ",
    "ﻉ": "ع",
    "ﻊ": "ع",
    "ﻋ": "ع",
    "ﻌ": "ع",
    "ﻍ": "غ",
    "ﻎ": "غ",
    "ﻏ": "غ",
    "ﻐ": "غ",
    "ﻑ": "ف",
    "ﻒ": "ف",
    "ﻓ": "ف",
    "ﻔ": "ف",
    "ﻕ": "ق",
    "ﻖ": "ق",
    "ﻗ": "ق",
    "ﻘ": "ق",
    "ﻙ": "ک",
    "ﻚ": "ک",
    "ﻛ": "ک",
    "ﻜ": "ک",
    "ﻝ": "ل",
    "ﻞ": "ل",
    "ﻟ": "ل",
    "ﻠ": "ل",
    "ﻡ": "م",
    "ﻢ": "م",
    "ﻣ": "م",
    "ﻤ": "م",
    "ﻥ": "ن",
    "ﻦ": "ن",
    "ﻧ": "ن",
    "ﻨ": "ن",
    "ﻩ": "ه",
    "ﻪ": "ه",
    "ﻫ": "ه",
    "ﻬ": "ه",
    "ﻭ": "و",
    "ﻮ": "و",
    "ﻯ": "ی",
    "ﻰ": "ی",
    "ﻱ": "ی",
    "ﻲ": "ی",
    "ﻳ": "ی",
    "ﻴ": "ی",
    "ﻵ": "لا",
    "ﻶ": "لا",
    "ﻷ": "لا",
    "ﻸ": "لا",
    "ﻹ": "لا",
    "ﻺ": "لا",
    "ﻻ": "لا",
    "ﻼ": "لا",
}

valid_chars = [
    " ",
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z",
    "a",
    "b",
    "c",
    "d",
    "e",
    "f",
    "g",
    "h",
    "i",
    "j",
    "k",
    "l",
    "m",
    "n",
    "o",
    "p",
    "q",
    "r",
    "s",
    "t",
    "u",
    "v",
    "w",
    "x",
    "y",
    "z",
    "آ",
    "ئ",
    "ا",
    "ب",
    "ت",
    "ث",
    "ج",
    "ح",
    "خ",
    "د",
    "ذ",
    "ر",
    "ز",
    "س",
    "ش",
    "ص",
    "ض",
    "ط",
    "ظ",
    "ع",
    "غ",
    "ف",
    "ق",
    "ل",
    "م",
    "ن",
    "ه",
    "و",
    "پ",
    "چ",
    "ژ",
    "ک",
    "گ",
    "ی",
]


def _replace_rep(t):
    "Replace repetitions at the character level: ccc -> c"

    def __replace_rep(m):
        c, cc = m.groups()
        return f"{c}"

    re_rep = re.compile(r"(\S)(\1{2,})")
    return re_rep.sub(__replace_rep, t)


def _replace_wrep(t):
    "Replace word repetitions: word word word -> word"

    def __replace_wrep(m):
        c, cc = m.groups()
        return f"{c}"

    re_wrep = re.compile(r"(\b\w+\W+)(\1{2,})")
    return re_wrep.sub(__replace_wrep, t)


def _normalize_text(x):
    """normalize a sentence"""

    x = str(x)
    x = parsivar_normalizer.normalize(x)  # apply `parsivar` normalizations
    x = re.sub(r"[\u200c\r\n]", " ", x)  # remove half space and new line characters
    x = x.lower()
    x = "".join(
        [char_mappings[xx] if xx in char_mappings else xx for xx in x]
    )  # substitue bad characters with appropriate ones
    x = re.sub(
        r"[^{}]".format("".join(valid_chars)), " ", x
    )  # just keep valid characters and substitue others with space
    x = re.sub(r"[a-z]+", r" \g<0> ", x)  # put space around words and numbers
    x = re.sub(r"[0-9]+", r" \g<0> ", x)  # put space around words and numbers
    x = re.sub(r"\s+", " ", x)  # remove more than one white spaces with space
    x = _replace_rep(x)
    x = _replace_wrep(x)
    return x.strip()


def normalize_texts(X, use_tqdm=False):
    """normalize list of sentences"""

    if use_tqdm:
        X = [_normalize_text(x) for x in tqdm(X)]
    else:
        X = [_normalize_text(x) for x in X]
    return X

In [ ]:
class JSONListWriter:
    """
    auxilary class to write list of dictionaries into json file.
    each item in one line.
    """

    def __init__(self, file_path):
        self.fd = None
        self.file_path = file_path
        self.delimiter = "\n"

    def open(self):
        self.fd = open(self.file_path, "w")
        self.first_item_written = False
        return self

    def close(self):
        self.fd.close()
        self.fd = None

    def write_item(self, obj):
        if self.first_item_written:
            self.fd.write(self.delimiter)
        self.fd.write(json.dumps(obj))
        self.first_item_written = True

    def __enter__(self):
        return self.open()

    def __exit__(self, type, value, traceback):
        self.close()

## Preprocess

In [32]:
spark.stop()